In [1]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import altair as alt

In [2]:
def get_fiis_url(full=True,my_fiis=[]):
    r = requests.get('https://fiis.com.br/lista-de-fundos-imobiliarios/')
    soup = BeautifulSoup(r.text, 'html.parser')
    fii_url_list = []
    fii_html_list = soup.find(id='items-wrapper').find_all('a')
    
    if full:
        for fii_url in fii_html_list:
            fii_url_list.append(fii_url.get('href'))
 
        return fii_url_list
    else:
        for fii in fii_html_list:
            fii_name = fii.contents[1].contents[0]
            if fii_name in my_fiis:
                fii_url_list.append(fii.get('href'))
        
        return fii_url_list

In [3]:
def get_fii_news(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    fii_news = {}
    fii_news['news'] = []
    
    news_block = soup.find(id='news--wrapper').find_all('li')
    for news in news_block:
        news_dict = {}
        
        news_date = news.find(class_='date').contents[0]
        if len(news.find(class_='title')) == 1:
            news_title = news.find(class_='title').contents[0]
        else:
            news_title = ''
            for piece in news.find(class_='title'):
                if str(piece) != '<br/>':
                    news_title = news_title+f'{piece} '
                    
            news_title = news_title.rstrip()
            
        if news.find('a').get('href') != 'javascript:;':
            news_link_url = news.find('a').get('href')
        else:
            news_link_url = 'no_link'
        
        news_dict['date'] = news_date
        news_dict['title'] = news_title
        news_dict['link'] = news_link_url
        fii_news['news'].append(news_dict)
    
    
    return fii_news

In [4]:
def get_fii_info(url,news=False):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    fii_info = {}
    
    fii_kpis = soup.find(id='informations--indexes').find_all(class_='item')
    fii_admin_top = soup.find(id='informations--admin').find(class_='top-content')
    fii_admin_bottom = soup.find(id='informations--admin').find(class_='bottom-content').find_all(class_='item')   
    fii_basic = soup.find(id='informations--basic').find_all(class_='wrapper')
    fii_basic_1_items = fii_basic[0].find_all(class_='item')
    fii_basic_2_items = fii_basic[1].find_all(class_='item')
    
   
    fii_info["fiiCode"] = soup.find(id='fund-ticker').contents[0]
    fii_info["fiiFullName"] = soup.find(id='fund-name').contents[0].rstrip().lstrip()
    fii_info["currentValue"] = float(soup.find(class_='item quotation').find(class_='value').contents[0].replace(',','.'))
    
    fii_info["admin"] = {}
    fii_info["admin"]["name"] = fii_admin_top.find(class_='administrator-name').contents[0]
    fii_info["admin"]["cnpj"] = fii_admin_top.find(class_='administrator-doc').contents[0]
    fii_info["admin"]["phone"] = fii_admin_bottom[0].find(class_='value').contents[0]
    fii_info["admin"]["email"] = fii_admin_bottom[1].find(class_='value').find('a').contents[0].contents[0]
    fii_info["admin"]["site"] = fii_admin_bottom[2].find(class_='value').find('a').contents[0]
    
    
    fii_info["fiiBasicInfo"] = {}
    fii_info["fiiBasicInfo"]["stockName"] = fii_basic_1_items[0].find(class_='value').contents[0]
    fii_info["fiiBasicInfo"]["fiiType"] = fii_basic_1_items[1].find(class_='value').contents[0]
    fii_info["fiiBasicInfo"]["typeANBIMA"] = fii_basic_1_items[2].find(class_='value').contents[0]
    fii_info["fiiBasicInfo"]["CVM"] = fii_basic_1_items[3].find(class_='value').contents[0]
    fii_info["fiiBasicInfo"]["numberOfQuotas"] = int(fii_basic_2_items[0].find(class_='value').contents[0].replace('.',''))
    fii_info["fiiBasicInfo"]["numberOfQuotaHolders"] = int(fii_basic_2_items[1].find(class_='value').contents[0].replace('.',''))
    fii_info["fiiBasicInfo"]["fiiCNPJ"] = fii_basic_2_items[2].find(class_='value').contents[0]
    
    
    fii_info["DY"] = float(fii_kpis[0].find(class_='value').contents[0].replace(',','.'))
    
    fii_info["lastPayment"] = {}
    fii_info["lastPayment"]["currency"] = fii_kpis[1].find(class_='value').find(class_='currency').contents[0]
    fii_info["lastPayment"]["value"] = float(fii_kpis[1].find(class_='value').contents[1].replace(',','.'))
    
    fii_info["netPatrimony"] = {}
    fii_info["netPatrimony"]["currency"] = fii_kpis[2].find(class_='value').find(class_='currency').contents[0]
    fii_info["netPatrimony"]["value"] = fii_kpis[2].find(class_='value').contents[1]
   
    fii_info["valuePerQuota"] = {}
    fii_info["valuePerQuota"]["currency"] = fii_kpis[3].find(class_='value').find(class_='currency').contents[0]
    fii_info["valuePerQuota"]["value"] = float(fii_kpis[3].find(class_='value').contents[1].replace(',','.'))
        
    if news:
        fii_info['news'] = get_fii_news(url)

    return fii_info

In [5]:
def fix_na_value(value):
    if value.get('data-order') == '-9999999999' or value.get('data-order') == '9999999999':
        return None
    else:
        return float(value.get('data-order'))

def get_fiis_attributes_table():
    url = 'https://www.fundsexplorer.com.br/ranking'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')


    fiis_table_dict = []

    table_rows = soup.find_all('tr')

    for table_row in table_rows:
        fii_row_dict = {}
        attributes = table_row.find_all('td')

        if len(attributes) > 0:
            fii_row_dict["code"] = attributes[0].find('a').contents[0]
            fii_row_dict["url"] = attributes[0].find('a').get('href')
            fii_row_dict["type"] = attributes[1].contents[0]
            fii_row_dict["current_price"] = fix_na_value(attributes[2])
            fii_row_dict["dividend"] = fix_na_value(attributes[3])

            fii_row_dict["dy_current"] = fix_na_value(attributes[5])
            fii_row_dict["dy_3m_sum"] = fix_na_value(attributes[6])
            fii_row_dict["dy_6m_sum"] = fix_na_value(attributes[7])
            fii_row_dict["dy_12m_sum"] = fix_na_value(attributes[8])
            fii_row_dict["dy_3m_avg"] = fix_na_value(attributes[9])
            fii_row_dict["dy_6m_avg"] = fix_na_value(attributes[10])
            fii_row_dict["dy_12m_avg"] = fix_na_value(attributes[11])
            fii_row_dict["dy_annual"] = fix_na_value(attributes[12])

            fii_row_dict["price_variation"] = fix_na_value(attributes[13])

            fii_row_dict["profitability_period"] = fix_na_value(attributes[14])
            fii_row_dict["profitability_sum_ytd"] = fix_na_value(attributes[15])


            fii_row_dict["patrimony_net_value"] = fix_na_value(attributes[16])
            fii_row_dict["patrimony_vpa"] = fix_na_value(attributes[17])
            fii_row_dict["patrimony_p_vpa"] = fix_na_value(attributes[18])
            fii_row_dict["patrimony_dy"] = fix_na_value(attributes[19])
            fii_row_dict["patrimony_variation"] = fix_na_value(attributes[20])

            fii_row_dict["patrimony_profitability_period"] = fix_na_value(attributes[21])
            fii_row_dict["patrimony_profitability_sum_ytd"] = fix_na_value(attributes[22])

            fii_row_dict["vacancy_physical"] = fix_na_value(attributes[23])
            fii_row_dict["vacancy_financial"] = fix_na_value(attributes[24])


            fii_row_dict["realty"] = int(attributes[25].get('data-order'))


            fiis_table_dict.append(fii_row_dict)
    return fiis_table_dict

In [6]:
def original_red_box(dataframe):
    plot = alt.Chart().mark_circle(size=60).encode(
        x='dy_12m_sum',
        y='patrimony_p_vpa',
        color='type',
        tooltip=['code', 'type', 'current_price', 'dy_current', 'patrimony_p_vpa', 'dy_12m_sum'],
        size='current_price'
    ).interactive()


    x_line = alt.Chart().mark_rule(color='red').encode(
        x='red_x:Q',
        size=alt.value(1)
    )

    y_line = alt.Chart().mark_rule(color='red').encode(
        y='red_y:Q',
        size=alt.value(1)
    )

    return alt.layer(
        plot, x_line, y_line,
        data=dataframe
    ).transform_calculate(
        red_x="8",
        red_y="1.2"
    )


In [7]:
# with open("../data/fiis_table.json", "w") as outfile:  
#     json.dump(get_fiis_attributes_table(), outfile)

In [8]:
my_fiis_json = "../data/my_fiis.json"


with open(my_fiis_json) as f:
  my_fiis_dict = json.load(f)

In [9]:
fiis_df = pd.DataFrame.from_dict(get_fiis_attributes_table())
my_fiis_df = fiis_df[fiis_df["code"].isin(my_fiis_dict["fiis"])]
fiis_df.describe

<bound method NDFrame.describe of        code            url                 type  current_price  dividend  \
0    FIVN11  /funds/fivn11            Shoppings           4.56     331.0   
1    BZLI11  /funds/bzli11  Títulos e Val. Mob.          14.80   80926.0   
2    XTED11  /funds/xted11   Lajes Corporativas           7.28    3427.0   
3    ALMI11  /funds/almi11   Lajes Corporativas        1195.15     105.0   
4    DOMC11  /funds/domc11   Lajes Corporativas         475.89       5.0   
..      ...            ...                  ...            ...       ...   
178  FCFL11  /funds/fcfl11               Outros         108.78    1930.0   
179  XPML11  /funds/xpml11            Shoppings         109.45  101425.0   
180  ALZR11  /funds/alzr11            Logística         128.26    9091.0   
181  FLRP11  /funds/flrp11            Shoppings        1394.00      14.0   
182  CNES11  /funds/cnes11   Lajes Corporativas          64.15     263.0   

     dy_current  dy_3m_sum  dy_6m_sum  dy_12m_sum  dy

In [109]:
red_box_df = fiis_df[(fiis_df["patrimony_p_vpa"]<=1.5)
                        & (fiis_df["dy_12m_sum"]>=8)
                        & (fiis_df["vacancy_physical"] < 20)
                        & (fiis_df["dy_current"] >= 0.5)
                        & (fiis_df["price_variation"] >= -10)].sort_values(ascending=True,by=["dy_current"])
red_box_df.head(100)

,code,url,type,current_price,dividend,dy_current,dy_3m_sum,dy_6m_sum,dy_12m_sum,dy_3m_avg,...,patrimony_net_value,patrimony_vpa,patrimony_p_vpa,patrimony_dy,patrimony_variation,patrimony_profitability_period,patrimony_profitability_sum_ytd,vacancy_physical,vacancy_financial,realty
136,MBRF11,/funds/mbrf11,Outros,1247.99,39.0,0.708048,2.101055,4.301006,8.338698,0.700352,...,1.306979e+08,1285.587038,0.970755,0.700069,0.021242,0.721460,5.120336,0.0,NaN,2
122,BBRC11,/funds/bbrc11,Outros,147.30,1087.0,0.770713,2.135733,4.161412,8.246298,0.711911,...,1.748209e+08,109.950267,1.339697,0.584426,-0.457060,0.124695,5.555142,0.0,0.0,20
132,RECT11,/funds/rect11,Híbrido,96.70,31170.0,0.838249,2.454543,5.015913,10.643082,0.818181,...,6.711422e+08,96.237028,1.004811,0.841672,0.021212,0.863062,2.419275,9.7,NaN,7
97,SPTW11,/funds/sptw11,Lajes Corporativas,79.93,3580.0,1.017639,2.695123,5.223750,10.140092,0.898374,...,1.367343e+08,76.048001,1.051047,0.986219,-0.781013,0.197504,3.466296,0.0,NaN,1
40,NSLU11,/funds/nslu11,Hospital,272.32,2289.0,1.433346,3.561200,5.710940,9.799170,1.187067,...,2.183788e+08,190.757177,1.427574,NaN,NaN,NaN,NaN,0.0,NaN,1
65,XPCM11,/funds/xpcm11,Lajes Corporativas,57.44,11955.0,1.626425,4.318868,9.127177,16.694642,1.439623,...,1.488364e+08,61.640940,0.931848,1.573629,0.587805,2.170684,10.008919,0.0,NaN,1


In [110]:
red_box_plot = original_red_box(red_box_df)
red_box_plot

alt.LayerChart(...)

In [12]:
my_fiis_df = my_fiis_df.sort_values(ascending=True,by=["code"])
my_fiis_df.head(100)

,code,url,type,current_price,dividend,dy_current,dy_3m_sum,dy_6m_sum,dy_12m_sum,dy_3m_avg,...,patrimony_net_value,patrimony_vpa,patrimony_p_vpa,patrimony_dy,patrimony_variation,patrimony_profitability_period,patrimony_profitability_sum_ytd,vacancy_physical,vacancy_financial,realty
145,BCFF11,/funds/bcff11,Títulos e Val. Mob.,88.70,47037.0,0.532544,1.407692,2.918359,6.244774,0.469231,...,1.721999e+09,83.742706,1.059197,0.537360,-1.562956,-1.033994,-19.106697,NaN,NaN,0
95,BCRI11,/funds/bcri11,Títulos e Val. Mob.,108.15,5455.0,0.694698,2.104932,4.408450,8.088260,0.701644,...,3.147348e+08,105.622841,1.023926,0.719541,-0.428728,0.287728,5.165827,NaN,NaN,0
121,BPFF11,/funds/bpff11,Títulos e Val. Mob.,85.05,5392.0,0.606061,1.688411,3.591117,7.962888,0.562804,...,2.496559e+08,93.544026,0.909198,0.577268,3.536230,4.133912,-10.032665,NaN,NaN,0
90,CBOP11,/funds/cbop11,Lajes Corporativas,74.45,1254.0,0.680650,1.695262,3.336538,6.793236,0.565087,...,1.215940e+08,85.932182,0.866381,0.570217,0.050741,0.621248,4.166529,10.37,10.87,1
130,CEOC11,/funds/ceoc11,Lajes Corporativas,83.00,1654.0,0.687775,1.952845,3.923734,7.606762,0.650948,...,1.780693e+08,98.072208,0.846315,0.582776,-0.049699,0.532787,3.694782,0.00,NaN,1
147,FEXC11,/funds/fexc11,Títulos e Val. Mob.,102.00,2787.0,0.582298,2.379832,4.892762,8.308096,0.793277,...,2.413455e+08,98.768992,1.032713,0.607478,-0.116830,0.489938,3.391963,NaN,NaN,0
59,HFOF11,/funds/hfof11,Títulos e Val. Mob.,113.50,23545.0,0.429185,2.190335,4.260027,8.411303,0.730112,...,1.668276e+09,98.973620,1.146770,0.505185,-3.430960,-2.943107,-4.551709,NaN,NaN,0
50,HGCR11,/funds/hgcr11,Títulos e Val. Mob.,103.49,22012.0,0.479138,1.460895,3.216844,6.508232,0.486965,...,1.308826e+09,105.785514,0.978300,0.453748,0.356137,0.811501,3.500952,NaN,NaN,0
157,IRDM11,/funds/irdm11,Títulos e Val. Mob.,115.97,41037.0,0.732849,2.182713,4.348447,9.025880,0.727571,...,1.030185e+09,97.105321,1.194270,0.871672,-1.481923,-0.623168,-1.623605,NaN,NaN,0
151,JRDM11,/funds/jrdm11,Shoppings,75.32,83.0,0.215822,0.215822,0.607616,3.897005,0.071941,...,2.281562e+08,90.431679,0.832894,0.178332,0.018122,0.196486,0.637825,2.21,NaN,1


In [13]:
my_fiis_plot = original_red_box(my_fiis_df)
my_fiis_plot

alt.LayerChart(...)

In [14]:
my_fiis_df[['current_price','dy_current','dividend',
            'dy_3m_sum','dy_6m_sum','dy_12m_sum',
            'dy_3m_avg','dy_6m_avg','dy_12m_avg',
            'patrimony_p_vpa',
            'vacancy_physical','vacancy_financial']].describe()

,current_price,dy_current,dividend,dy_3m_sum,dy_6m_sum,dy_12m_sum,dy_3m_avg,dy_6m_avg,dy_12m_avg,patrimony_p_vpa,vacancy_physical,vacancy_financial
count,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,7.000000,1.00
mean,89.327273,0.632083,43353.227273,1.902985,3.962549,7.537581,0.634328,0.660425,0.628132,1.004431,3.082857,10.87
std,24.091657,0.270001,113200.606973,0.727540,1.479681,2.896780,0.242513,0.246613,0.241398,0.090710,3.801599,NaN
min,10.570000,0.215822,83.000000,0.215822,0.607616,0.000000,0.071941,0.101269,0.000000,0.832894,0.000000,10.87
25%,80.697500,0.490458,5357.500000,1.592119,3.369027,6.701180,0.530706,0.561504,0.558432,0.956027,0.000000,10.87
50%,96.255000,0.600903,11839.000000,1.882923,3.902832,7.532707,0.627641,0.650472,0.627726,1.020978,2.210000,10.87
75%,103.382500,0.685994,38849.500000,2.099377,4.326342,8.385501,0.699792,0.721057,0.698792,1.054868,4.500000,10.87
max,115.970000,1.626425,543002.000000,4.318868,9.127177,16.694642,1.439623,1.521196,1.391220,1.194270,10.370000,10.87


# FIIs in crysis
Observation of indicators of crysis over FIIs

## Dividend = 0%

In [15]:
zero_dy_df = fiis_df[fiis_df["dy_current"] == 0]
zero_dy_df.head(100)

,code,url,type,current_price,dividend,dy_current,dy_3m_sum,dy_6m_sum,dy_12m_sum,dy_3m_avg,...,patrimony_net_value,patrimony_vpa,patrimony_p_vpa,patrimony_dy,patrimony_variation,patrimony_profitability_period,patrimony_profitability_sum_ytd,vacancy_physical,vacancy_financial,realty
0,FIVN11,/funds/fivn11,Shoppings,4.56,331.0,0.0,0.000000,0.000000,0.000000,0.000000,...,7.155951e+07,7.602499,0.599803,NaN,NaN,NaN,NaN,56.00,NaN,1
1,BZLI11,/funds/bzli11,Títulos e Val. Mob.,14.80,80926.0,0.0,0.000000,0.000000,0.000000,0.000000,...,4.280817e+08,11.864763,1.247391,NaN,NaN,NaN,NaN,NaN,NaN,0
2,XTED11,/funds/xted11,Lajes Corporativas,7.28,3427.0,0.0,0.000000,0.000000,0.000000,0.000000,...,2.629879e+07,14.671105,0.496213,NaN,NaN,NaN,NaN,100.00,100.0,1
3,ALMI11,/funds/almi11,Lajes Corporativas,1195.15,105.0,0.0,0.000000,0.000000,0.000000,0.000000,...,2.205547e+08,1983.816265,0.602450,NaN,NaN,NaN,NaN,71.19,NaN,1
4,DOMC11,/funds/domc11,Lajes Corporativas,475.89,5.0,0.0,0.000000,0.000000,0.000000,0.000000,...,2.441484e+08,999.052229,0.476341,NaN,NaN,NaN,NaN,23.44,NaN,1
5,PABY11,/funds/paby11,Híbrido,15.75,13.0,0.0,0.000000,0.000000,0.000000,0.000000,...,-2.771783e+06,-3.654777,-4.309428,NaN,NaN,NaN,NaN,NaN,NaN,1
7,RBCB11,/funds/rbcb11,Outros,19.00,161.0,0.0,0.285486,4.380644,14.619389,0.095162,...,4.007139e+04,0.754640,25.177568,NaN,NaN,NaN,NaN,NaN,NaN,0
14,VSHO11,/funds/vsho11,Shoppings,87.00,85.0,0.0,1.175079,2.880465,6.508591,0.391693,...,2.060428e+08,98.171737,0.886202,NaN,NaN,NaN,NaN,5.67,NaN,3
15,HTMX11,/funds/htmx11,Hotel,118.00,1936.0,0.0,0.801222,2.196960,5.413236,0.267074,...,1.822786e+08,145.008284,0.813747,NaN,NaN,NaN,NaN,93.00,NaN,23
54,NVIF11B,/funds/nvif11b,Outros,NaN,NaN,0.0,0.000000,0.000000,0.000000,0.000000,...,6.375428e+07,188.957565,NaN,0.470557,-1.079215,-0.613736,4.682578,NaN,NaN,0


## High Vacancy
### Pyshical or Financial vacancy over 30%

In [16]:
high_vacancy_df = fiis_df[(fiis_df["vacancy_physical"] >= 50) | (fiis_df["vacancy_financial"] >= 30.0)]
high_vacancy_df.head(100)

,code,url,type,current_price,dividend,dy_current,dy_3m_sum,dy_6m_sum,dy_12m_sum,dy_3m_avg,...,patrimony_net_value,patrimony_vpa,patrimony_p_vpa,patrimony_dy,patrimony_variation,patrimony_profitability_period,patrimony_profitability_sum_ytd,vacancy_physical,vacancy_financial,realty
0,FIVN11,/funds/fivn11,Shoppings,4.56,331.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,7.155951e+07,7.602499,0.599803,NaN,NaN,NaN,NaN,56.00,NaN,1
2,XTED11,/funds/xted11,Lajes Corporativas,7.28,3427.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.629879e+07,14.671105,0.496213,NaN,NaN,NaN,NaN,100.00,100.0,1
3,ALMI11,/funds/almi11,Lajes Corporativas,1195.15,105.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.205547e+08,1983.816265,0.602450,NaN,NaN,NaN,NaN,71.19,NaN,1
6,BBVJ11,/funds/bbvj11,Lajes Corporativas,57.50,51.0,0.611224,0.742696,0.935552,0.961387,0.247565,...,1.558767e+08,57.519084,0.999668,NaN,NaN,NaN,NaN,70.40,NaN,1
15,HTMX11,/funds/htmx11,Hotel,118.00,1936.0,0.000000,0.801222,2.196960,5.413236,0.267074,...,1.822786e+08,145.008284,0.813747,NaN,NaN,NaN,NaN,93.00,NaN,23
52,BBFI11B,/funds/bbfi11b,Lajes Corporativas,2299.90,184.0,1.102383,2.926600,6.083454,11.492221,0.975533,...,4.080523e+08,3138.864010,0.732717,0.806892,0.033097,0.840256,5.457429,72.20,NaN,2


In [40]:
bars_physical = alt.Chart(high_vacancy_df).mark_bar().encode(
    x='vacancy_physical:Q',
    y='code:N',
    color='type:N',
    tooltip=['code', 'type', 'current_price', 'dy_current', 'patrimony_p_vpa', 'dy_12m_sum']
)

bars_financial = alt.Chart(high_vacancy_df).mark_bar().encode(
    x='vacancy_financial:Q',
    y='code:N',
    color='type:N',
    tooltip=['code', 'type', 'current_price', 'dy_current', 'patrimony_p_vpa', 'dy_12m_sum']
)

text_physical = bars_physical.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text='vacancy_physical:Q'
)

text_financial = bars_financial.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text='vacancy_financial:Q'
)

In [41]:
(bars_physical + text_physical).properties(height=150)

alt.LayerChart(...)

In [42]:
(bars_financial + text_financial).properties(height=150)

alt.LayerChart(...)

## Price Variation (going down)
### Price closure about the past two months

In [62]:
price_variation_down_df = fiis_df[fiis_df["price_variation"] < 0].sort_values(ascending=True,by=["price_variation"])
price_variation_down_df.head(200)

,code,url,type,current_price,dividend,dy_current,dy_3m_sum,dy_6m_sum,dy_12m_sum,dy_3m_avg,...,patrimony_net_value,patrimony_vpa,patrimony_p_vpa,patrimony_dy,patrimony_variation,patrimony_profitability_period,patrimony_profitability_sum_ytd,vacancy_physical,vacancy_financial,realty
55,CXCE11B,/funds/cxce11b,Outros,99.97,329.0,0.815658,2.404465,4.976711,9.762065,0.801488,...,1.477014e+08,86.226339,1.159391,0.944058,-96.003368,-95.965638,-95.764890,0.00,NaN,1
7,RBCB11,/funds/rbcb11,Outros,19.00,161.0,0.000000,0.285486,4.380644,14.619389,0.095162,...,4.007139e+04,0.754640,25.177568,NaN,NaN,NaN,NaN,NaN,NaN,0
88,BCIA11,/funds/bcia11,Títulos e Val. Mob.,150.08,2462.0,0.546847,1.364476,2.792440,8.378984,0.454825,...,1.947031e+08,115.235343,1.302378,0.650842,-3.614192,-2.986873,-12.804406,NaN,NaN,0
15,HTMX11,/funds/htmx11,Hotel,118.00,1936.0,0.000000,0.801222,2.196960,5.413236,0.267074,...,1.822786e+08,145.008284,0.813747,NaN,NaN,NaN,NaN,93.00,NaN,23
14,VSHO11,/funds/vsho11,Shoppings,87.00,85.0,0.000000,1.175079,2.880465,6.508591,0.391693,...,2.060428e+08,98.171737,0.886202,NaN,NaN,NaN,NaN,5.67,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,VISC11,/funds/visc11,Shoppings,106.90,32760.0,0.174503,0.589250,1.666410,5.169957,0.196417,...,1.732376e+09,121.391415,0.880622,0.148281,-0.150044,-0.001986,1.809007,7.20,NaN,13
28,HGRU11,/funds/hgru11,Híbrido,129.60,49367.0,0.532290,1.649898,3.388883,6.761052,0.549966,...,1.351576e+09,107.833020,1.201858,0.630605,0.021159,0.651897,4.043823,0.00,0.0,15
153,BMLC11B,/funds/bmlc11b,Lajes Corporativas,92.54,349.0,0.568481,1.836196,3.589978,6.438499,0.612065,...,9.385991e+07,94.009858,0.984365,0.574400,-0.102471,0.471340,2.403200,1.70,NaN,2
60,RDPD11,/funds/rdpd11,Títulos e Val. Mob.,82.69,3.0,0.382244,1.536214,2.806892,5.772290,0.512071,...,1.049616e+08,97.363157,0.849295,0.318396,-0.249835,0.067766,2.745592,NaN,NaN,0


In [66]:
bars = alt.Chart(price_variation_down_df).mark_bar().encode(
    x='price_variation:Q',
    # y='code:N',
    y=alt.Y('code:N', sort='x'),
    # color='type:N',
    tooltip=['code', 'type', 'current_price', 'price_variation','dy_current', 'patrimony_p_vpa', 'dy_12m_sum']
)

text = bars.mark_text(
    align='right',
    baseline='middle',
    dx=3
).encode(
    text='price_variation:Q'
)

(bars + text).properties(height=2000, width=500)

alt.LayerChart(...)

# High performance FIIs
Observation of kpis poiting to high performaces FIIs

## RedBox Method (by GuiaInvest)
### V/PA vs DY last 12 months

## Price Variation (going up + nice DY + low v/pa)
### Price closure about the past two months

In [102]:
price_variation_up_df = fiis_df[(fiis_df["price_variation"] > 0)
                            & (fiis_df["dy_current"] > 0.7)
                            & (fiis_df["patrimony_p_vpa"] < 1.2)].sort_values(ascending=False,by=["dy_current"])
price_variation_up_df.head(200)

,code,url,type,current_price,dividend,dy_current,dy_3m_sum,dy_6m_sum,dy_12m_sum,dy_3m_avg,...,patrimony_net_value,patrimony_vpa,patrimony_p_vpa,patrimony_dy,patrimony_variation,patrimony_profitability_period,patrimony_profitability_sum_ytd,vacancy_physical,vacancy_financial,realty
9,KNRE11,/funds/knre11,Residencial,4.10,6700.0,3.919859,5.108468,5.108468,9.245911,1.702823,...,7.687063e+07,4.144604,0.989238,NaN,NaN,NaN,NaN,NaN,NaN,0
97,SPTW11,/funds/sptw11,Lajes Corporativas,79.93,3580.0,1.017639,2.695123,5.223750,10.140092,0.898374,...,1.367343e+08,76.048001,1.051047,0.986219,-0.781013,0.197504,3.466296,0.0,NaN,1
152,HCTR11,/funds/hctr11,Outros,128.40,10948.0,0.918439,2.680838,6.322899,12.121855,0.893613,...,2.253778e+08,117.149554,1.096035,0.998723,0.503771,1.507525,10.318771,NaN,NaN,0
91,HABT11,/funds/habt11,Títulos e Val. Mob.,105.55,12217.0,0.893377,2.226659,4.597862,0.000000,0.742220,...,3.896342e+08,101.770121,1.037141,0.903998,0.425444,1.333289,6.333727,NaN,NaN,0
170,MORE11,/funds/more11,Títulos e Val. Mob.,105.50,2932.0,0.786730,1.674618,0.000000,0.000000,0.558206,...,6.050049e+07,103.624758,1.018096,0.579012,5.587008,6.198370,5.536705,NaN,NaN,0
118,PORD11,/funds/pord11,Títulos e Val. Mob.,99.37,3100.0,0.731707,2.344982,4.429567,11.852559,0.781661,...,1.987969e+08,100.766625,0.986140,0.719241,-0.291304,0.425842,1.158794,NaN,NaN,0
89,BARI11,/funds/bari11,Títulos e Val. Mob.,103.50,62057.0,0.713855,2.034328,4.168580,8.758549,0.678109,...,2.214571e+08,99.121845,1.044169,0.716290,-0.132692,0.582647,3.946564,NaN,NaN,0
162,VGIP11,/funds/vgip11,Outros,98.90,2037.0,0.704935,2.118021,0.000000,0.000000,0.706007,...,7.141013e+07,95.722625,1.033194,0.687175,0.051923,0.739454,-1.808196,NaN,NaN,0


In [103]:
bars = alt.Chart(price_variation_up_df).mark_bar().encode(
    x='price_variation:Q',
    # y='code:N',
    y=alt.Y('code:N', sort='-x'),
    # color='type:N',
    tooltip=['code', 'type', 'current_price', 'price_variation','dy_current', 'patrimony_p_vpa', 'dy_12m_sum']
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text='price_variation:Q'
)

(bars + text).properties(height=200, width=500)

alt.LayerChart(...)

# KPIs per FII types
Analyses based on the FIIs types

## Top 3 average price variation (going up)

## Top 3 average DY

## Top 3 average vacancy